In [ ]:
PATH_TO_SAVED_MODEL = "/home/exported-models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/saved_model/"
PATH_TO_LABELS = "/home/label_map.pbtxt"

In [ ]:
import time
import pandas as pd
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
%matplotlib inline

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [ ]:
import glob
types = ('*.png', '*.jpg') # the tuple of file types
files_grabbed = []
for files in types:
    files_grabbed.extend(glob.glob('/home/evaluation_metrics/model_validation/validation_data/test/'+ files))

In [ ]:
IMAGE_PATHS = files_grabbed
IMAGE_PATHS

In [ ]:
# IMAGE_PATHS=['/home/evaluation_metrics/model_validation/validation_data/test/In_Everest Cuisine_01_18.jpg']

In [ ]:
IMAGE_PATHS[0].split('/')[-1]

In [ ]:
# import json
# with open("/home/evaluation_metrics/model_validation/validation_data/dictionary_label.txt","r") as f:
#     labels = f.read().replace("\n","")
# labels = json.loads(labels)

In [ ]:
test_pred = pd.DataFrame()
#     columns = ['filename', 'width','height','class','xmin','ymin','xmax','ymax','score'])

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:
    try:
        print('Running inference for {}... '.format(image_path.split('/')[-1]), end='')



        image_np = load_image_into_numpy_array(image_path)
        
    
        if len(image_np.shape)==2:
            image_np = image_np.reshape((image_np.shape[0],image_np.shape[1],1))

        # Things to try:
        # Flip horizontally
        #         image_np = np.fliplr(image_np).copy()

        # Convert image to grayscale
        image_np = np.tile(
            np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)
        
#         print(image_np.shape)
        height, width, color = image_np.shape
#         print("height : ",height)
#         print("width : ",width)

        # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
        input_tensor = tf.convert_to_tensor(image_np)
        # The model expects a batch of images, so add an axis with `tf.newaxis`.
        input_tensor = input_tensor[tf.newaxis, ...]

        # input_tensor = np.expand_dims(image_np, 0)
        detections = detect_fn(input_tensor)

        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                       for key, value in detections.items()}


        for i in range(num_detections):
            ll = []
            ll.append(image_path)
            ll.append(width)
            ll.append(height)
            ll.append(detections['detection_boxes'][i].tolist())
            ll.append(category_index[detections['detection_classes'][i]]['name'])
            ll.append(detections['detection_scores'][i])

            pre_df =pd.DataFrame([{'filename':ll[0],
                                   'width':ll[1],
                                   'height':ll[2],
                                   'class':ll[4],
                                   'xmin':ll[3][1] * width,
                                   'ymin':ll[3][0] * height,
                                   'xmax':ll[3][3] * width,
                                   'ymax':ll[3][2] * height,
                                   'score':ll[5]}])
            test_pred = pd.concat([test_pred,pre_df],axis=0)
        print('Done')
    except:
        print("image has issue :" + image_path.split('/')[-1])

In [ ]:
test_pred

In [ ]:
# test_pred = test_pred[test_pred['score']>=0.5]
# test_pred = test_pred[test_pred['class']=='Item']

In [ ]:
# test_pred

In [ ]:
test_pred['filename'].unique()

In [ ]:
test_pred.to_csv("/home/evaluation_metrics/model_validation/validation_data/test_pred/test_pred.csv",index=False)